In [33]:
import nltk
import csv
from nltk.tokenize import word_tokenize
from string import punctuation
import re
import spacy
from spacy import displacy
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import stanfordnlp
import pandas as pd
import csv
from stanfordnlp.server import CoreNLPClient


STOP_WORDS = set(ENGLISH_STOP_WORDS)
STOP_WORDS.remove('no')
STOP_WORDS.remove('not')
#STOP_WORDS.remove('of')
print(STOP_WORDS)

{'thru', 'becomes', 'herein', 'call', 'so', 'due', 'together', 'do', 'anyhow', 'then', 'mill', 'against', 'please', 'formerly', 'us', 'through', 'nobody', 'much', 'fill', 'already', 'them', 'etc', 'however', 'one', 'two', 'it', 'over', 'still', 'elsewhere', 'under', 'whenever', 'another', 'must', 'all', 'eg', 'themselves', 'towards', 'around', 'whole', 'that', 'via', 'can', 'ie', 'she', 'whereupon', 'before', 'during', 'indeed', 'itself', 'except', 'each', 'every', 'put', 'even', 'serious', 'neither', 'thus', 'empty', 'four', 'out', 'also', 'have', 'are', 'in', 'back', 'such', 'forty', 'detail', 'somehow', 'several', 'perhaps', 'wherein', 'am', 'a', 'latter', 'side', 'ours', 'yourself', 'had', 'name', 'else', 'he', 'toward', 'about', 'of', 'becoming', 'alone', 'his', 'off', 'last', 'either', 'former', 'very', 'some', 'thereby', 'whoever', 'beforehand', 'first', 'my', 'seemed', 'third', 'fire', 'though', 'amongst', 'seems', 'often', 'eleven', 'within', 'twelve', 'more', 'de', 'everyone'

In [2]:
nlp = stanfordnlp.Pipeline(processors = "tokenize,mwt,pos,depparse")

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\schaa\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\schaa\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\schaa\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\schaa\\stanfordnlp_resources\\en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': 'C:\\Users\\schaa\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Done loading processors!
---


In [5]:
abbreviation_path = 'Metrics/concept_abbreviation.txt'
with open(abbreviation_path, 'r') as dictionary:
    abbreviations_dic = {}
    for line in dictionary:
        if len(line)>1:
            line=line.split('-',1)
            abbreviations_dic[line[0].rstrip().lower()] = line[1].lstrip().lower()
            
digit_pattern = '\d+'
date_pattern = r"[\d]{1,2}/[\d]{1,2}/[\d]{2,4}"
time_pattern = r"[\d]{1,2}:[\d]{1,2}:[\d]{1,2}"
DIGIT_SIGN = '/DIGIT'
DATE_SIGN = 'DATE'
TIME_SIGN = 'TIME'
apas_error = '&apos;'

##find way to replace dates by today for example -> see what they depend on!!

In [7]:
filepath = 'clinical-13.txt'

text = []
with open(filepath, 'r') as file:
    data = file.read()
    
data = re.sub(date_pattern, DATE_SIGN, data)
data = re.sub(time_pattern, TIME_SIGN, data)
#data = re.sub(digit_pattern, DIGIT_SIGN, data)
data = re.sub(apas_error, "'", data)
print(data)

Admission Date :
2013-05-09
Discharge Date :
2013-06-02
Date of Birth :
1966-03-19
Sex :
M
Service :
Scott
HISTORY OF PRESENT ILLNESS :
The patient is a 47 year old male , with end stage liver disease secondary to hepatitis C cirrhosis diagnosed about 5 years prior to admission .
The patient had undergone treatment with interferon and Ribavirin .
He had been admitted to the St. Margaret 's Center for Women & Infants multiple times early in 2013 for management of encephalopathy and ascites .
The patient had been discharged from the St. Margaret 's Center for Women & Infants on 2013-05-06 , but was readmitted on 2013-05-09 when noted to have worsening renal function .
The patient 's serum creatinine on the day of discharge , on 2013-05-06 , was 1.9 , but was noted to increase to
3.2 on 2013-05-08 , and was further elevated to 3.6 on 2013-05-09 .
The patient was admitted with concern for hepatorenal syndrome .
PAST MEDICAL HISTORY :
1. Hepatitis C cirrhosis for which the patient was on th

In [8]:
tokens = word_tokenize(data)
for i in range(len(tokens)):
    if tokens[i].lower() in abbreviations_dic:
        for j,n in enumerate(abbreviations_dic[tokens[i].lower()].split()):
            tokens.insert(i+j+1,n)
        del tokens[i]

print(tokens)

['Admission', 'Date', ':', '2013-05-09', 'Discharge', 'Date', ':', '2013-06-02', 'Date', 'of', 'Birth', ':', '1966-03-19', 'Sex', ':', 'M', 'Service', ':', 'Scott', 'HISTORY', 'OF', 'PRESENT', 'ILLNESS', ':', 'The', 'patient', 'is', 'mibi', '-', 'adenosine', 'mibi', '47', 'year', 'old', 'male', ',', 'with', 'end', 'stage', 'liver', 'disease', 'secondary', 'to', 'hepatitis', 'with', 'cirrhosis', 'diagnosed', 'about', '5', 'years', 'prior', 'to', 'admission', '.', 'The', 'patient', 'had', 'undergone', 'treatment', 'with', 'interferon', 'axillary', 'node', 'dissection', 'Ribavirin', '.', 'He', 'had', 'been', 'admitted', 'to', 'the', 'St.', 'Margaret', "'s", 'Center', 'for', 'Women', '&', 'Infants', 'multiple', 'times', 'early', 'in', '2013', 'for', 'management', 'of', 'encephalopathy', 'axillary', 'node', 'dissection', 'ascites', '.', 'The', 'patient', 'had', 'been', 'discharged', 'from', 'the', 'St.', 'Margaret', "'s", 'Center', 'for', 'Women', '&', 'Infants', 'optic', 'nerve;', 'over', 

In [ ]:
# tagged_review = nltk.pos_tag(tokens[20:])
# tagged_review

In [34]:
#doc = nlp(data)
with CoreNLPClient(annotators=['tokenize','ssplit', 'mwt', 'pos', 'depparse'], timeout=30000, memory='16G') as client:
    # submit the request to the server
    ann = client.annotate(text)

AssertionError: Please define $CORENLP_HOME to be location of your CoreNLP Java checkout

In [11]:
pos_dict = {
'CC': 'coordinating conjunction','CD': 'cardinal digit','DT': 'determiner',
'EX': 'existential there (like: \"there is\" ... think of it like \"there exists\")',
'FW': 'foreign word','IN':  'preposition/subordinating conjunction','JJ': 'adjective \'big\'',
'JJR': 'adjective, comparative \'bigger\'','JJS': 'adjective, superlative \'biggest\'',
'LS': 'list marker 1)','MD': 'modal could, will','NN': 'noun, singular \'desk\'',
'NNS': 'noun plural \'desks\'','NNP': 'proper noun, singular \'Harrison\'',
'NNPS': 'proper noun, plural \'Americans\'','PDT': 'predeterminer \'all the kids\'',
'POS': 'possessive ending parent\'s','PRP': 'personal pronoun I, he, she',
'PRP$': 'possessive pronoun my, his, hers','RB': 'adverb very, silently,',
'RBR': 'adverb, comparative better','RBS': 'adverb, superlative best',
'RP': 'particle give up','TO': 'to go \'to\' the store.','UH': 'interjection errrrrrrrm',
'VB': 'verb, base form take','VBD': 'verb, past tense took',
'VBG': 'verb, gerund/present participle taking','VBN': 'verb, past participle taken',
'VBP': 'verb, sing. present, non-3d take','VBZ': 'verb, 3rd person sing. present takes',
'WDT': 'wh-determiner which','WP': 'wh-pronoun who, what','WP$': 'possessive wh-pronoun whose',
'WRB': 'wh-abverb where, when','QF' : 'quantifier, bahut, thoda, kam (Hindi)','VM' : 'main verb',
'PSP' : 'postposition, common in indian langs','DEM' : 'demonstrative, common in indian langs'
}

#extract parts of speech
def extract_pos(sentences):
    parsed_text = {'word':[], 'pos':[], 'exp':[]}
    for sent in sentences:
        for wrd in sent.words:
            if wrd.pos in pos_dict.keys():
                pos_exp = pos_dict[wrd.pos]
            else:
                pos_exp = 'NA'
            parsed_text['word'].append(wrd.text)
            parsed_text['pos'].append(wrd.pos)
            parsed_text['exp'].append(pos_exp)
    #return a dataframe of pos and text
    return pd.DataFrame(parsed_text)

#extract pos
extract_pos(doc.sentences[10:12])

,word,pos,exp
0,The,DT,determiner
1,patient,NN,"noun, singular 'desk'"
2,was,VBD,"verb, past tense took"
3,admitted,VBN,"verb, past participle taken"
4,with,IN,preposition/subordinating conjunction
5,concern,NN,"noun, singular 'desk'"
6,for,IN,preposition/subordinating conjunction
7,hepatorenal,JJ,adjective 'big'
8,syndrome,NN,"noun, singular 'desk'"
9,.,.,NA


In [12]:
for word in doc.sentences[5].words:
    print(word.text)

The
patient
is
a
47
year
old
male
,
with
end
stage
liver
disease
secondary
to
hepatitis
C
cirrhosis
diagnosed
about
5
years
prior
to
admission
.


In [13]:
for sent in doc.sentences:
    sent.print_dependencies()
    print('-----------------')
    
    
##have to check both for words that the concept is dependent on and for words that depend on the concept
##e.g. negative for tumours -> tumours depends on negative here

('Admission', '2', 'compound')
('Date', '0', 'root')
(':', '2', 'punct')
('2013-05-09', '2', 'appos')
-----------------
('Discharge', '2', 'compound')
('Date', '0', 'root')
(':', '2', 'punct')
('2013-06-02', '2', 'appos')
-----------------
('Date', '0', 'root')
('of', '3', 'case')
('Birth', '1', 'nmod')
(':', '1', 'punct')
('1966-03-19', '1', 'appos')
('Sex', '1', 'parataxis')
(':', '1', 'punct')
-----------------
('M', '2', 'compound')
('Service', '0', 'root')
(':', '2', 'punct')
('Scott', '2', 'appos')
-----------------
('HISTORY', '0', 'root')
('OF', '4', 'case')
('PRESENT', '4', 'amod')
('ILLNESS', '1', 'nmod')
(':', '1', 'punct')
-----------------
('The', '2', 'det')
('patient', '8', 'nsubj')
('is', '8', 'cop')
('a', '8', 'det')
('47', '6', 'nummod')
('year', '7', 'obl:npmod')
('old', '8', 'amod')
('male', '0', 'root')
(',', '8', 'punct')
('with', '15', 'case')
('end', '12', 'compound')
('stage', '15', 'compound')
('liver', '14', 'compound')
('disease', '15', 'compound')
('seconda

-----------------
('Part', '20', 'nsubj')
('of', '6', 'case')
('the', '6', 'det')
('patient', '6', 'compound')
("'s", '6', 'compound')
('chart', '1', 'nmod')
('from', '9', 'case')
('the', '9', 'det')
('period', '6', 'nmod')
('2013-', '11', 'compound')
('05-09', '9', 'nummod')
('to', '13', 'case')
('2013', '9', 'nmod')
('-', '15', 'case')
('05', '13', 'nmod')
('-', '17', 'case')
('23', '15', 'nmod')
('is', '20', 'cop')
('currently', '20', 'advmod')
('unavailable', '0', 'root')
(',', '27', 'punct')
('and', '27', 'cc')
('this', '24', 'det')
('dictation', '27', 'nsubj')
('will', '27', 'aux')
('mainly', '27', 'advmod')
('cover', '20', 'conj')
('the', '29', 'det')
('period', '27', 'obj')
('from', '31', 'case')
('2013-', '29', 'nmod')
('05', '29', 'nummod')
('-', '34', 'case')
('23', '32', 'nmod')
('to', '36', 'case')
('2013', '27', 'obl')
('-', '38', 'case')
('06-02', '36', 'nmod')
(')', '36', 'punct')
-----------------
('As', '3', 'mark')
('previously', '3', 'advmod')
('mentioned', '17', 'a

('while', '19', 'mark')
('still', '19', 'advmod')
('intubated', '11', 'advcl')
('following', '22', 'case')
('the', '22', 'det')
('procedure', '19', 'obl')
('.', '3', 'punct')
-----------------
('The', '2', 'det')
('patient', '3', 'nsubj')
('underwent', '0', 'root')
('an', '6', 'det')
('uncomplicated', '6', 'amod')
('recovery', '3', 'obj')
('in', '11', 'case')
('the', '11', 'det')
('intensive', '11', 'amod')
('care', '11', 'compound')
('unit', '3', 'obl')
('.', '3', 'punct')
-----------------
('By', '3', 'case')
('postop', '3', 'amod')
('day', '9', 'obl')
('1', '3', 'nummod')
(',', '9', 'punct')
('the', '7', 'det')
('patient', '9', 'nsubj')
('was', '9', 'cop')
('awake', '0', 'root')
(',', '9', 'punct')
('in', '13', 'case')
('no', '13', 'det')
('distress', '9', 'obl')
(',', '16', 'punct')
('and', '16', 'cc')
('appeared', '9', 'conj')
('lucid', '16', 'xcomp')
('prior', '20', 'case')
('to', '18', 'fixed')
('extubation', '16', 'obl')
('.', '9', 'punct')
-----------------
('The', '2', 'det')

('4', '2', 'nummod')
('.', '7', 'punct')
('Fluconazole', '7', 'compound')
('200', '5', 'nummod')
('mg', '7', 'compound')
('po', '7', 'compound')
('qd', '0', 'root')
('.', '1', 'punct')
-----------------
('5', '2', 'nummod')
('.', '7', 'punct')
('CellCept', '7', 'compound')
('1', '3', 'nummod')
('gm', '7', 'compound')
('po', '7', 'compound')
('bid', '0', 'root')
('.', '7', 'punct')
-----------------
('6', '2', 'nummod')
('.', '7', 'punct')
('Prednisone', '7', 'compound')
('20', '5', 'nummod')
('mg', '7', 'compound')
('po', '7', 'compound')
('qd', '0', 'root')
('.', '1', 'punct')
-----------------
('7', '4', 'nummod')
('.', '4', 'punct')
('Dilaudid', '4', 'compound')
('prn', '0', 'root')
('.', '4', 'punct')
-----------------
('8', '2', 'nummod')
('.', '7', 'punct')
('Neoral', '7', 'compound')
('500', '5', 'nummod')
('mg', '7', 'compound')
('po', '7', 'compound')
('bid', '0', 'root')
('.', '7', 'punct')
-----------------
('9', '2', 'nummod')
('.', '7', 'punct')
('Valcyte', '7', 'compound'

In [31]:
def analyse_deps(sentence):
    dependencies = list()
    for word in sentence.words:
        if word.governor == 0:
            dependencies.append([0, word.text])
        else:
            dependencies.append([word.governor-1, word.text])
    return dependencies

###now: try to extract sentences where concept is present -> put this into pipeline!!

In [32]:
with open('clinical-13_diseases.csv') as csvfile:
    csvdata = csv.reader(csvfile)
    next(csvdata, None)
    conc_beginning = []
    conc_end = []
    range_txt = []
    concept = []
    for row in csvdata:
        conc_beginning.append(int(row[12]))
        conc_end.append(int(row[11]))
        range_txt.append(row[13])
        concept.append(row[3])

def get_sentence(range_txt, conc_beginning, conc_end):
    txt_length = 0
    for i, sent in enumerate(doc.sentences):
        for word in sent.words:
            if conc_beginning <= txt_length <= conc_end:
                sentence = ' '.join([word.text for word in doc.sentences[i].words])
                if range_txt not in sentence:
                    continue
                else:
                    return i
            txt_length += len(word.text) + 1
    return 0
    

    
def get_word_dependents(doc, range_txt, conc_beginning, conc_end):
        index = get_sentence(range_txt, conc_beginning, conc_end)
        depend = analyse_deps(doc.sentences[index])
        words = [word.text for word in doc.sentences[index].words]
        pos = [word.pos for word in doc.sentences[index].words]
        range_words = range_txt.split(' ')
        print(words, range_words)
        for i in range(len(words)):
            flag = 0
            for j in range(len(range_words)):
                if range_words[j].lower() == words[i+j].lower():
                    flag += 1
                    if flag >= len(range_words):
                        dep_indices = [k for k, value in enumerate(depend) if value[0] in range(i,i+j+1)]
                        dep_indices2 = [depend[k][0] for k in range (i,i+j+1)]
                        dep_indices += dep_indices2
                        print(dep_indices)
                        return [[words[l] for l in dep_indices if words[l] not in range_words], [pos[l] for l in dep_indices if words[l] not in range_words]]

dependents = list()
for i in range(len(range_txt)):
    dependents.append((range_txt[i], get_word_dependents(doc, range_txt[i], conc_beginning[i], conc_end[i])))
    
print(dependents)

for dependent in dependents:
    if dependent[1] is None:
        continue
    removal = list()
    for i, (wrd, pos) in enumerate(zip(dependent[1][0], dependent[1][1])):
        if pos in ['VBD', 'VB', 'VBG', 'VBP', 'VBN','EX', 'CC', 'HYPH', '.', ',', 'LS', 'PRP', 'PRP$', 'WDT', 'WP', 'WP$', 'WRB']:
            removal.append([wrd,pos])
        elif wrd.lower() in STOP_WORDS:
            removal.append([wrd,pos])
    for j in removal:
        dependent[1][0].remove(j[0])
        dependent[1][1].remove(j[1])
        
            
for dependent in dependents:
    print('Word:', dependent[0], '\t- Dependents:', dependent[1])

##chunking

['The', 'patient', 'is', 'a', '47', 'year', 'old', 'male', ',', 'with', 'end', 'stage', 'liver', 'disease', 'secondary', 'to', 'hepatitis', 'C', 'cirrhosis', 'diagnosed', 'about', '5', 'years', 'prior', 'to', 'admission', '.'] ['end', 'stage', 'liver', 'disease']
[10, 12, 11, 14, 13, 14]
['The', 'patient', 'is', 'a', '47', 'year', 'old', 'male', ',', 'with', 'end', 'stage', 'liver', 'disease', 'secondary', 'to', 'hepatitis', 'C', 'cirrhosis', 'diagnosed', 'about', '5', 'years', 'prior', 'to', 'admission', '.'] ['hepatitis', 'C']
[16, 17, 18]
['The', 'patient', 'is', 'a', '47', 'year', 'old', 'male', ',', 'with', 'end', 'stage', 'liver', 'disease', 'secondary', 'to', 'hepatitis', 'C', 'cirrhosis', 'diagnosed', 'about', '5', 'years', 'prior', 'to', 'admission', '.'] ['cirrhosis']
[15, 17, 19, 14]
['He', 'had', 'been', 'admitted', 'to', 'the', 'St.', 'Margaret', "'s", 'Center', 'for', 'Women', '&', 'Infants', 'multiple', 'times', 'early', 'in', '2013', 'for', 'management', 'of', 'encephal

In [ ]:
position = sentence.index(range_txt[i])